# evaluators

> Class to evaluate results

In [ ]:
#| default_exp evaluators

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|exporti 

import os
from typing import List, Iterator, Dict, Union, Tuple, Optional
import lir
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from tqdm import tqdm
from collections import defaultdict

from lr_video_face.experiments import Experiment, ExperimentalSetup
from lr_video_face.plots import *



In [ ]:
#|export

class ExperimentEvaluator:
    def __init__(self, 
    experiment: Experiment,
    cllr_expert_per_year: Dict[int, list],
    results: Dict[str, list],
    save_plots: bool = True
    ):

        self.experiment = experiment
        self.cllr_expert_per_year = cllr_expert_per_year
        self.results = results
        self.save_plots = save_plots
        self.experiment_directory = self.experiment.output_dir
        self.cllr_auto_per_year = self.get_cllr_auto_per_year()
    

    def make_plots(self):
        plot_lr_distributions(self.results, self.experiment_directory, self.save_plots)
        plot_ROC_curve(self.results, self.experiment_directory, self.save_plots)
        plot_tippett(self.results, self.experiment_directory, self.save_plots)
        plot_ece(self.results, self.experiment_directory, self.save_plots)
        plot_cllr(self.results, self.experiment_directory, self.experiment.enfsi_years, 
            self.cllr_expert_per_year, self.cllr_auto_per_year, self.experiment.embeddingModel, self.save_plots)


    def get_cllr_auto_per_year(self):
        years = [pair.first.year for pair in self.results["test_pairs"]]
        lrs_predicted = self.results["lrs_predicted"]
        y_test = self.results["y_test"]

        data_per_year = zip(years, lrs_predicted, y_test)

        lrs_predicted_per_year = defaultdict(list)
        y_test_per_year = defaultdict(list)

        for year, lr, y in data_per_year:
            lrs_predicted_per_year[year].append(lr)
            y_test_per_year[year].append(y)

        cllr_auto_per_year = {}

        for year in np.unique(years):
            cllr_auto_per_year[year] = lir.metrics.cllr(np.asarray(lrs_predicted_per_year[year]),
                                                        np.asarray(y_test_per_year[year]))

        return cllr_auto_per_year


In [ ]:
#| export

class GlobalEvaluator:

    def __init__(self, 
    experiments: ExperimentalSetup,
    save_plots: bool = True
    ):

        self.experiments = experiments
        self.save_plots = save_plots
        self.experiment_evaluators = self.get_experiment_evaluators(self.experiments)

    
    @staticmethod
    def get_experiment_evaluators(experiments: ExperimentalSetup) -> List[ExperimentEvaluator]:
        evaluators = []
        for experiment in tqdm(experiments):
            results = experiment.perform()
            evaluation = ExperimentEvaluator(experiment=experiment, results=results, cllr_expert_per_year=experiments.cllr_expert_per_year)
            evaluators.append(evaluation)
        return evaluators

    def make_experiment_plots(self):
        for evaluator in self.experiment_evaluators:
            evaluator.make_plots()


    def make_global_plot(self):
        experiment_df = pd.DataFrame(
            columns=['Year', 'Filters', 'Detector', 'Embedding Model', 'Calibrator', 'Cllr'])
        filters = self.experiments.filters + self.experiments.face_image_filters
        str_filters = ",".join(filters)
        for evaluator in self.experiment_evaluators:

            if isinstance(evaluator.experiment.calibrator, IsotonicCalibrator):
                calibrator_name = 'Isotonic Calibrator'
            else:
                calibrator_name = str(evaluator.experiment.calibrator)

            for year in evaluator.experiment.enfsi_years:
                experiment_df = experiment_df.append(
                    {'Year': year, 'Filters': str_filters, 'Detector': evaluator.experiment.detector,
                     'Embedding Model': evaluator.experiment.embeddingModel,
                     'Calibrator': calibrator_name.split("(")[0],
                     'Cllr': evaluator.cllr_auto_per_year[year],
                     }, ignore_index=True)

        for year in self.experiments.enfsi_years:
            for detector in self.experiments.detectors:
                for calibrator in self.experiments.calibrators:

                    if isinstance(calibrator, IsotonicCalibrator):
                        calibrator_name = 'Isotonic Calibrator'
                    else:
                        calibrator_name = str(calibrator)

                    experiment_df = experiment_df.append(
                        {'Year': year, 'Filters': str_filters, 'Detector': detector,
                         'Embedding Model': "Average Participants",
                         'Calibrator': calibrator_name.split("(")[0],
                         'Cllr': mean(self.experiments.cllr_expert_per_year[year])
                         }, ignore_index=True)

        experiment_df.to_excel(os.path.join(self.experiments.output_dir,
                                             f'results_file_ES{self.experiments.embedding_model_as_scorer}.xlsx'))

        sns.set_style("whitegrid")
        sns.catplot(data=experiment_df, x="Year", y="Cllr", hue="Embedding Model", row="Calibrator", col="Detector")
        savefig = os.path.join(self.experiments.output_dir,
                               f"cllr_summary_ES{self.experiments.embedding_model_as_scorer}")
        plt.savefig(savefig)
        plt.close() 


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()